We want to solve:
$$0=h(k)-(u')^{-1}\left( \beta u'\left( h\left((f(k) +(1- \delta)k- h(k)\right) \right) f'\left(f(k) +(1- \delta)k - h(k) \right) \right)$$
where $c=h(k)$ using projection on Chebychev polynomials.

In [1]:
using Parameters
using DOIHS
using ValidatedNumerics
using Plots; gr()

Plots.GRBackend()

The model and its functions are:

In [2]:
@with_kw immutable GrowthModel
    β::Float64                           # discount factor
    δ::Float64                           # depreciation
    α::Float64                           # capital share
    A::Float64                           # productivity
    γ::Float64                           # RRA
end

model = GrowthModel(β = 0.95, δ = 0.05, α = 0.3, A = 1, γ = 2)

function steady_state_k(model::GrowthModel)
    @unpack β, α, A, δ = model
    ((1/β-(1-δ))/(A*α))^(1/(α-1))
end

function f(model::GrowthModel, k)
    @unpack α, A, δ = model
    A*k.^α
end

function f_prime(model::GrowthModel, k)
    @unpack α, A, δ = model
    A*α*k.^(α-1)
end

u_crra_prime(c, γ) = c.^-γ
u_crra_prime_inv(u, γ) = u.^(-1/γ)

u_crra_prime_inv (generic function with 1 method)

In [3]:
k_stst = steady_state_k(model)

4.628988089138438

The basis of the interpolation is:

In [4]:
basis = IntervalAB((0.2*k_stst)..(2*k_stst), ChebyshevBasis(10, extrapolate=true))

DOIHS.ChebyshevBasis(10,false,true) on [0.925797, 9.25798]

In [483]:
@unpack β, α, A, δ, γ = model
Ψ = basis_matrix(basis)
K = collocation_points(basis)
a = [-1, 0, 0, 0, 0, 0, 0, 0, 0, 0];

In [484]:
stop = false

@time while !stop
    
    a_old = a
    Y = u_crra_prime_inv(β*u_crra_prime(basis_matrix(basis, f(model,K) + (1-δ)*K - Ψ*a)*a, γ) .* 
            (f_prime(model, f(model,K) + (1-δ)*K - Ψ*a) + 1-δ), γ)
    a = Ψ \ Y #with a square Ψ the regression really is just the solution of a linear equation
    
    if maximum(abs(u_crra_prime(Y, γ)./u_crra_prime(Ψ*a_old, γ) - 1)) < 1e-10
        stop = true
    end
    
end

  0.022582 seconds (27.03 k allocations: 2.376 MB)


In [485]:
ks = linspace(basis, 200)

plot(ks,basis_matrix(basis, ks)*a, ylab="consumption", xlab="capital")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 2.5 
 
 
 5.0 
 
 
 7.5 
 
 
 1.0 
 
 
 1.5 
 
 
 capital 
 
 
 consumption 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 45.8815,369.674 48.6462,365.963 51.4109,362.329 54.1756,358.768 56.9404,355.276 59.7051,351.851 62.4698,348.489 65.2346,345.187 67.9993,341.943 70.764,338.752 
 73.5288,335.614 76.2935,332.526 79.0582,329.484 81.823,326.488 84.5877,323.536 87.3524,320.624 90.1172,317.753 92.8819,314.919 95.6466,312.121 98.4113,309.359 
 101.176,306.63 103.941,303.933 106.706,301.267 109.47,298.63 112.235,296.023 115,293.443 117.764,290.889 120.529,288.362 123.294,285.859 126.059,283.381 
 128.823,280.925 131.588,278.493 134.353,276.082 137.118,273.693 139.882,271.324 142.647,268.975 145.412,266.646 148.177,264.336 150.941,262.045 153.706,259.771 
 156.471,257.515 159.235,255.277 162,253.055 164.765,250.85 167.53,248.66 170.294,246.487 173.059,244.329 175.824,242.186 178.589,240.057 181.353,237.943 
 184.118,235.844 186.883,233.758 189.647,231.685 192.412,229.626 195.177,227.58 197.942,225.547 200.706,223.527 203.471,221.519 206.236,219.523 209.001,217.539 
 211.765,215.567 214.53,213.606 217.295,211.657 220.06,209.718 222.824,207.791 225.589,205.874 228.354,203.968 231.118,202.072 233.883,200.186 236.648,198.31 
 239.413,196.444 242.177,194.588 244.942,192.741 247.707,190.903 250.472,189.075 253.236,187.255 256.001,185.444 258.766,183.642 261.53,181.848 264.295,180.063 
 267.06,178.286 269.825,176.517 272.589,174.756 275.354,173.002 278.119,171.257 280.884,169.518 283.648,167.788 286.413,166.064 289.178,164.348 291.943,162.639 
 294.707,160.937 297.472,159.242 300.237,157.553 303.001,155.871 305.766,154.196 308.531,152.527 311.296,150.864 314.06,149.208 316.825,147.558 319.59,145.914 
 322.355,144.276 325.119,142.644 327.884,141.018 330.649,139.398 333.414,137.783 336.178,136.174 338.943,134.571 341.708,132.973 344.472,131.381 347.237,129.794 
 350.002,128.212 352.767,126.636 355.531,125.065 358.296,123.499 361.061,121.939 363.826,120.383 366.59,118.833 369.355,117.287 372.12,115.747 374.884,114.211 
 377.649,112.68 380.414,111.154 383.179,109.633 385.943,108.117 388.708,106.605 391.473,105.098 394.238,103.596 397.002,102.098 399.767,100.605 402.532,99.1165 
 405.297,97.6323 408.061,96.1526 410.826,94.6772 413.591,93.2062 416.355,91.7396 419.12,90.2772 421.885,88.8191 424.65,87.3651 427.414,85.9153 430.179,84.4697 
 432.944,83.0281 435.709,81.5906 438.473,80.157 441.238,78.7275 444.003,77.3018 446.767,75.88 449.532,74.4621 452.297,73.0479 455.062,71.6375 457.826,70.2309 
 460.591,68.8278 463.356,67.4285 466.121,66.0327 468.885,64.6404 471.65,63.2517 474.415,61.8664 477.18,60.4846 479.944,59.1061 482.709,57.7311 485.474,56.3593 
 488.238,54.9908 491.003,53.6256 493.768,52.2636 496.533,50.9048 499.297,49.5492 502.062,48.1967 504.827,46.8473 507.592,45.501 510.356,44.1577 513.121,42.8175 
 515.886,41.4802 518.651,40.146 521.415,38.8148 524.18,37.4865 526.945,36.1612 529.709,34.8388 532.474,33.5193 535.239,32.2027 538.004,30.8891 540.768,29.5783 
 543.533,28.2704 546.298,26.9653 549.063,25.6631 551.827,24.3638 554.592,23.0673 557.357,21.7736 560.121,20.4827 562.886,19.1946 565.651,17.9092 568.416,16.6265 
 571.18,15.3465 573.945,14.0692 576.71,12.7944 579.475,11.5222 582.239,10.2524 585.004,8.98498 587.769,7.71986 590.534,6.45691 593.298,5.196 596.063,3.93701 
 
 "/>
 
 
 
 
 y1